# Setup and Imports

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
from dotenv import load_dotenv
from numba import config

from src.tann.evolution.individual import predict_individual, initialize_individual, individual_actions
from src.tann.evolution.population import initialize_population, get_population_actions
from src.tann.network.neuron import compute_layer_indices
from src.tann.network.utils import get_layer_parameters

# Auto-reload modules
%load_ext autoreload
%autoreload 2

# Load environment and set working directory
load_dotenv()
root = Path(os.getenv("ROOT"))
os.chdir(root)

# Configuration

In [2]:
# Objective parameters
n_features = 1
output_size = 3
output_activation = 2
features = ["price"]
timestamp_col = "timestamp"

# Individual parameters
hidden_layers_sizes = [10, 10]
hidden_layers_activations = [1, 1, 1]

# Helpers
seed = 123
epoch_id = 0

# Numba configuration

# Load and Prepare Data

In [3]:
df = pl.read_parquet(root / 'data/mock.parquet')
df = df.with_columns(
    (pl.col('timestamp') // 30_000_000).alias('epoch'),
)

# Initialize Network Architecture

In [4]:
layer_sizes = [n_features] + hidden_layers_sizes + [output_size]
layer_activations = hidden_layers_activations + [output_activation]
previous_states = np.zeros(np.sum(layer_sizes[1:]), dtype=np.float64)
previous_time = df.filter(pl.col('epoch') == epoch_id).select('timestamp').to_series()[0]

# Compute layer indices for efficient processing
param_indices, neuron_indices = compute_layer_indices(layer_sizes)

# Initialize Individual

In [5]:
individual = initialize_individual(layer_sizes, seed=seed)

# Prepare Data for Testing

In [6]:
df_epoch = df.filter(pl.col("epoch") == epoch_id)

# Extract timestamps and feature values
timestamps = df_epoch[timestamp_col].to_numpy()
feature_values = df_epoch.select(features).to_numpy()

# Prepare single input for testing
current_time = df_epoch.select(timestamp_col).to_series()[0]
values = np.array([df_epoch.select(features).to_series()[0]], dtype=np.float64)
inputs = (current_time, values)

# Initialize Population

In [7]:
# Initialize population with He initialization for ReLU
pop_size = 100
population = initialize_population(pop_size, layer_sizes, seed=123, init_method="he")

# Initialize population states
population_states = np.zeros((pop_size, np.sum(layer_sizes[1:])), dtype=np.float64)

# Benchmark Population Actions

In [8]:
population_actions = get_population_actions(population, layer_sizes, layer_activations, population_states, previous_time, param_indices, neuron_indices, timestamps, feature_values)

In [9]:
df_epoch

timestamp,price,message_rate,epoch
i64,f64,f64,i64
3029,100.0,0.016667,0
3510,99.809521,0.033333,0
4616,99.623538,0.05,0
6077,99.43912,0.066667,0
8027,99.270446,0.083333,0
…,…,…,…
29979297,92.88041,0.25,0
29980497,92.864343,0.25,0
29997255,92.868848,0.183333,0


In [10]:
import pandas as pd
pd.DataFrame(population_actions.T).nunique()

0     1
1     1
2     1
3     1
4     1
     ..
95    1
96    1
97    1
98    1
99    1
Length: 100, dtype: int64

In [11]:
feature_values

array([[100.        ],
       [ 99.80952071],
       [ 99.6235381 ],
       ...,
       [ 92.86884801],
       [ 92.87140925],
       [ 92.85269645]], shape=(38949, 1))

# Trading Fitness Functions

# Trading Fitness Functions (Complete)

In [12]:
from numba import njit, prange

# Helper function for running maximum calculation
@njit(fastmath=True, cache=True)
def running_maximum(arr):
    """Calculate running maximum of an array."""
    n = len(arr)
    if n == 0:
        return arr
    
    running_max = np.zeros(n, dtype=np.float64)
    running_max[0] = arr[0]
    
    for i in range(1, n):
        running_max[i] = max(running_max[i-1], arr[i])
    
    return running_max


@njit(fastmath=True, cache=True)
def get_individual_returns_with_costs(actions, prices, transaction_cost=0.000):
    """
    Calculate returns for an individual with transaction costs.
    
    Actions:
    - 0: Sell (go short or close long)
    - 1: Hold (do nothing)
    - 2: Buy (go long or close short)
    
    Args:
        actions: Array of actions
        prices: Array of prices
        transaction_cost: Cost per trade as fraction (default 0.02% = 2 bps)
    
    Returns:
        returns: Array of returns per timestamp
        trades: Number of trades executed
        position_changes: Number of position changes
    """
    n_timestamps = len(actions)
    returns = np.zeros(n_timestamps, dtype=np.float64)
    position = 0  # -1: short, 0: flat, 1: long
    entry_price = 0.0
    trades = 0
    position_changes = 0
    
    for i in range(n_timestamps):
        action = actions[i]
        price = prices[i]
        old_position = position
        
        if position == 0:  # Flat
            if action == 2:  # Buy
                position = 1
                entry_price = price * (1 + transaction_cost)
                trades += 1
            elif action == 0:  # Sell
                position = -1
                entry_price = price * (1 - transaction_cost)
                trades += 1
                
        elif position == 1:  # Long
            if action == 0:  # Sell - close long
                exit_price = price * (1 - transaction_cost)
                returns[i] = (exit_price - entry_price) / entry_price
                position = 0
                entry_price = 0.0
                trades += 1
                
        else:  # position == -1 (Short)
            if action == 2:  # Buy - close short
                exit_price = price * (1 + transaction_cost)
                returns[i] = (entry_price - exit_price) / entry_price
                position = 0
                entry_price = 0.0
                trades += 1
        
        if old_position != position:
            position_changes += 1
    
    return returns, trades, position_changes


@njit(fastmath=True, cache=True)
def calculate_fitness_score_simple(returns, trades, position_changes):
    """
    Simple fitness score based on total return.
    
    Returns:
        fitness_score: Total compound return
        metrics: Tuple of (compound_return, trades, win_rate, avg_return, 
                          max_drawdown, sharpe, calmar)
    """
    # Handle no trades case
    if trades == 0:
        return 0.0, (0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0)
    
    # Calculate returns metrics
    total_return = np.sum(returns)
    compound_return = np.prod(1 + returns) - 1
    
    # Win rate
    winning_trades = np.sum(returns > 0)
    win_rate = winning_trades / trades
    
    # Average return per trade
    avg_return = total_return / trades
    
    # Maximum drawdown
    cumulative_returns = np.cumprod(1 + returns) - 1
    running_max = running_maximum(cumulative_returns)
    drawdowns = (cumulative_returns - running_max) / (1 + running_max)
    max_drawdown = np.min(drawdowns) if len(drawdowns) > 0 else 0.0
    
    # Sharpe ratio (annualized)
    returns_per_trade = returns[returns != 0]
    if len(returns_per_trade) > 1:
        returns_std = np.std(returns_per_trade)
        if returns_std > 0:
            sharpe = (avg_return) / returns_std * np.sqrt(252)
        else:
            sharpe = 0.0
    else:
        sharpe = 0.0
    
    # Calmar ratio (return / max drawdown)
    calmar = -compound_return / max_drawdown if max_drawdown < 0 else compound_return
    
    # Simple fitness: just compound return
    fitness_score = compound_return
    
    return fitness_score, (compound_return, trades, win_rate, avg_return, 
                          max_drawdown, sharpe, calmar)


@njit(fastmath=True, cache=True, parallel=True)
def get_population_fitness(population_actions, prices, transaction_cost=0.000):
    """
    Calculate fitness scores for entire population using simple total return.
    
    Args:
        population_actions: 2D array (pop_size, n_timestamps) of actions
        prices: 1D array of prices
        transaction_cost: Cost per trade (default 0.02%)
    
    Returns:
        fitness_scores: Array of fitness scores (total returns)
        population_metrics: 2D array of metrics for each individual
    """
    pop_size = population_actions.shape[0]
    fitness_scores = np.zeros(pop_size, dtype=np.float64)
    population_metrics = np.zeros((pop_size, 7), dtype=np.float64)
    
    for i in prange(pop_size):
        returns, trades, position_changes = get_individual_returns_with_costs(
            population_actions[i], prices, transaction_cost
        )
        
        fitness, metrics = calculate_fitness_score_simple(returns, trades, position_changes)
        fitness_scores[i] = fitness
        
        # Store metrics
        population_metrics[i, 0] = metrics[0]  # compound_return
        population_metrics[i, 1] = metrics[1]  # trades
        population_metrics[i, 2] = metrics[2]  # win_rate
        population_metrics[i, 3] = metrics[3]  # avg_return
        population_metrics[i, 4] = metrics[4]  # max_drawdown
        population_metrics[i, 5] = metrics[5]  # sharpe
        population_metrics[i, 6] = metrics[6]  # calmar
    
    return fitness_scores, population_metrics

In [13]:
# Test improved fitness functions
print("=== Testing Improved Fitness Functions ===")

# Extract prices as 1D array
prices = feature_values.flatten()

# Calculate fitness for population
fitness_scores, population_metrics = get_population_fitness(population_actions, prices)

# Find best performers by fitness score
best_indices = np.argsort(fitness_scores)[-10:][::-1]

print("\n=== Top 10 Performers by Fitness Score ===")
print(f"{'Rank':<5} {'ID':<5} {'Fitness':<10} {'Return':<10} {'Trades':<8} {'Win%':<8} {'Sharpe':<10} {'Calmar':<10} {'MaxDD':<10}")
print("-" * 90)

for rank, idx in enumerate(best_indices):
    metrics = population_metrics[idx]
    print(f"{rank+1:<5} {idx:<5} {fitness_scores[idx]:<10.4f} "
          f"{metrics[0]*100:<10.2f}% {int(metrics[1]):<8} {metrics[2]*100:<8.1f}% "
          f"{metrics[5]:<10.4f} {metrics[6]:<10.4f} {metrics[4]*100:<10.2f}%")

# Analyze population statistics
print("\n=== Population Statistics ===")
print(f"Average fitness score: {np.mean(fitness_scores):.4f}")
print(f"Average compound return: {np.mean(population_metrics[:, 0])*100:.2f}%")
print(f"Average number of trades: {np.mean(population_metrics[:, 1]):.1f}")
print(f"Average win rate: {np.mean(population_metrics[:, 2])*100:.1f}%")
print(f"Average Sharpe ratio: {np.mean(population_metrics[:, 5]):.4f}")

# Find individuals that actually traded
traded_mask = population_metrics[:, 1] > 0  # trades > 0
n_traded = np.sum(traded_mask)
print(f"\nIndividuals that made trades: {n_traded}/{pop_size}")

if n_traded > 0:
    print("\n=== Statistics for Trading Individuals ===")
    trading_metrics = population_metrics[traded_mask]
    print(f"Average return (traders only): {np.mean(trading_metrics[:, 0])*100:.2f}%")
    print(f"Average trades: {np.mean(trading_metrics[:, 1]):.1f}")
    print(f"Average win rate: {np.mean(trading_metrics[:, 2])*100:.1f}%")
    print(f"Best Sharpe ratio: {np.max(trading_metrics[:, 5]):.4f}")
    print(f"Best Calmar ratio: {np.max(trading_metrics[:, 6]):.4f}")

# Visualize best performer
best_idx = best_indices[0]
best_actions = population_actions[best_idx]

# Show action distribution for best performer
action_names = ['Sell', 'Hold', 'Buy']
unique, counts = np.unique(best_actions, return_counts=True)
print(f"\n=== Best Performer (ID {best_idx}) Action Distribution ===")
for action, count in zip(unique, counts):
    print(f"{action_names[action]}: {count} ({count/len(best_actions)*100:.1f}%)")

=== Testing Improved Fitness Functions ===

=== Top 10 Performers by Fitness Score ===
Rank  ID    Fitness    Return     Trades   Win%     Sharpe     Calmar     MaxDD     
------------------------------------------------------------------------------------------
1     99    0.0000     0.00      % 1        0.0     % 0.0000     0.0000     0.00      %
2     36    0.0000     0.00      % 1        0.0     % 0.0000     0.0000     0.00      %
3     26    0.0000     0.00      % 1        0.0     % 0.0000     0.0000     0.00      %
4     27    0.0000     0.00      % 0        0.0     % 0.0000     0.0000     0.00      %
5     28    0.0000     0.00      % 1        0.0     % 0.0000     0.0000     0.00      %
6     29    0.0000     0.00      % 1        0.0     % 0.0000     0.0000     0.00      %
7     30    0.0000     0.00      % 1        0.0     % 0.0000     0.0000     0.00      %
8     31    0.0000     0.00      % 1        0.0     % 0.0000     0.0000     0.00      %
9     32    0.0000     0.00      

# Simple Genetic Algorithm

In [14]:
@njit(fastmath=True, cache=True)
def mutate_individual(individual, mutation_rate=0.1, mutation_strength=0.1):
    """
    Apply random mutations to an individual.
    
    Args:
        individual: Network parameters to mutate
        mutation_rate: Probability of mutating each parameter
        mutation_strength: Standard deviation of mutation noise
    
    Returns:
        Mutated individual
    """
    mutated = individual.copy()
    n_params = len(individual)
    
    for i in range(n_params):
        if np.random.random() < mutation_rate:
            # Add Gaussian noise
            mutated[i] += np.random.normal(0, mutation_strength)
    
    return mutated


@njit(fastmath=True, cache=True)
def simple_genetic_algorithm(population, fitness_scores, elite_fraction=0.1, 
                           mutation_rate=0.1, mutation_strength=0.1):
    """
    Simple genetic algorithm: keep top 10%, regenerate rest from elite with mutations.
    
    Args:
        population: Current population (pop_size, n_params)
        fitness_scores: Fitness scores for each individual
        elite_fraction: Fraction of population to keep as elite
        mutation_rate: Probability of mutating each parameter
        mutation_strength: Strength of mutations
    
    Returns:
        New population
    """
    pop_size = population.shape[0]
    n_params = population.shape[1]
    n_elite = int(pop_size * elite_fraction)
    
    # Get indices of elite individuals (top performers)
    elite_indices = np.argsort(fitness_scores)[-n_elite:]
    
    # Create new population
    new_population = np.zeros((pop_size, n_params), dtype=np.float64)
    
    # Copy elite individuals
    for i in range(n_elite):
        new_population[i] = population[elite_indices[i]]
    
    # Fill rest of population with mutated copies of elite
    for i in range(n_elite, pop_size):
        # Randomly select parent from elite
        parent_idx = np.random.randint(0, n_elite)
        parent = new_population[parent_idx]
        
        # Create mutated offspring
        new_population[i] = mutate_individual(parent, mutation_rate, mutation_strength)
    
    return new_population


# Run genetic algorithm with simple fitness
def run_evolution_simple(initial_population, prices, scaled_features, timestamps, 
                        n_generations=20, elite_fraction=0.1):
    """
    Run genetic algorithm with simple total return fitness.
    """
    population = initial_population.copy()
    pop_size = population.shape[0]
    states = np.zeros((pop_size, np.sum(layer_sizes[1:])), dtype=np.float64)
    
    # Track best fitness over generations
    best_fitness_history = []
    avg_fitness_history = []
    best_trades_history = []
    
    print(f"Running genetic algorithm for {n_generations} generations...")
    print(f"Population size: {pop_size}, Elite size: {int(pop_size * elite_fraction)}")
    print(f"Fitness = Total Compound Return")
    print("-" * 80)
    
    for gen in range(n_generations):
        # Get actions for current population
        population_actions = get_population_actions(
            population, layer_sizes, layer_activations,
            states, 0, param_indices, neuron_indices,
            timestamps, scaled_features
        )
        
        # Calculate fitness (simple total return)
        fitness_scores, metrics = get_population_fitness(population_actions, prices)
        
        # Track statistics
        best_fitness = np.max(fitness_scores)
        avg_fitness = np.mean(fitness_scores)
        best_fitness_history.append(best_fitness)
        avg_fitness_history.append(avg_fitness)
        
        # Find best individual
        best_idx = np.argmax(fitness_scores)
        best_return = metrics[best_idx, 0] * 100  # Compound return
        best_trades = int(metrics[best_idx, 1])
        best_sharpe = metrics[best_idx, 5]
        best_trades_history.append(best_trades)
        
        print(f"Gen {gen+1:3d} | Best Return: {best_return:7.2f}% | "
              f"Avg Return: {avg_fitness*100:7.2f}% | "
              f"Best Trades: {best_trades:3d} | "
              f"Sharpe: {best_sharpe:6.3f}")
        
        # Evolve population
        population = simple_genetic_algorithm(
            population, fitness_scores, 
            elite_fraction=elite_fraction,
            mutation_rate=0.2,
            mutation_strength=0.2
        )
    
    return population, best_fitness_history, avg_fitness_history, best_trades_history


# Run the evolution with simple fitness
print("=== Starting Evolution with Simple Total Return Fitness ===\n")

# Use properly initialized population with scaled inputs
initial_pop = initialize_population(100, layer_sizes, seed=42, init_method="he")

# Prepare data
prices = feature_values.flatten()
scaled_features = feature_values / 100.0
reasonable_timestamps = np.arange(len(timestamps)) * 100

print(f"Data shapes:")
print(f"  Prices: {prices.shape}")
print(f"  Scaled features: {scaled_features.shape}")
print(f"  Timestamps: {reasonable_timestamps.shape}")
print()

# Run evolution with simple fitness
final_pop, best_history, avg_history, trades_history = run_evolution_simple(
    initial_pop, prices, scaled_features, reasonable_timestamps,
    n_generations=300,
    elite_fraction=0.2
)

=== Starting Evolution with Simple Total Return Fitness ===

Data shapes:
  Prices: (38949,)
  Scaled features: (38949, 1)
  Timestamps: (38949,)

Running genetic algorithm for 300 generations...
Population size: 100, Elite size: 20
Fitness = Total Compound Return
--------------------------------------------------------------------------------
Gen   1 | Best Return:    0.00% | Avg Return:    0.00% | Best Trades:   0 | Sharpe:  0.000
Gen   2 | Best Return:    0.00% | Avg Return:    0.00% | Best Trades:   1 | Sharpe:  0.000
Gen   3 | Best Return:   15.48% | Avg Return:    0.15% | Best Trades:  95 | Sharpe:  1.970
Gen   4 | Best Return:   23.27% | Avg Return:    0.46% | Best Trades:  31 | Sharpe:  2.109
Gen   5 | Best Return:   23.27% | Avg Return:    0.59% | Best Trades:  31 | Sharpe:  2.109
Gen   6 | Best Return:   25.69% | Avg Return:    1.22% | Best Trades:   5 | Sharpe:  6.387
Gen   7 | Best Return:   25.69% | Avg Return:    1.61% | Best Trades:   5 | Sharpe:  6.387
Gen   8 | Best Re

In [15]:
# Visualize evolution results with plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots for evolution progress
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Evolution Progress', 'Best Individual Performance'),
    horizontal_spacing=0.15
)

# Plot fitness over generations
generations = np.arange(1, len(best_history) + 1)
fig.add_trace(
    go.Scatter(x=generations, y=best_history, mode='lines', name='Best Fitness',
               line=dict(color='blue', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=generations, y=avg_history, mode='lines', name='Average Fitness',
               line=dict(color='red', width=2, dash='dash')),
    row=1, col=1
)

# Get final best individual
final_states = np.zeros((pop_size, np.sum(layer_sizes[1:])), dtype=np.float64)
final_actions = get_population_actions(
    final_pop, layer_sizes, layer_activations,
    final_states, 0, param_indices, neuron_indices,
    reasonable_timestamps, scaled_features
)
final_fitness, final_metrics = get_population_fitness(final_actions, prices)
best_final_idx = np.argmax(final_fitness)

# Calculate returns for visualization
returns, _, _ = get_individual_returns_with_costs(final_actions[best_final_idx], prices)
cumulative_returns = np.cumprod(1 + returns) - 1

# Plot cumulative returns
fig.add_trace(
    go.Scatter(x=list(range(len(cumulative_returns))), 
               y=cumulative_returns * 100, 
               mode='lines', 
               name='Cumulative Return',
               line=dict(color='green', width=1),
               fill='tozeroy',
               fillcolor='rgba(0,255,0,0.1)'),
    row=1, col=2
)

# Add zero line
fig.add_hline(y=0, line_dash="dot", line_color="black", opacity=0.5, row=1, col=2)

# Update layout
fig.update_xaxes(title_text="Generation", row=1, col=1)
fig.update_yaxes(title_text="Fitness Score", row=1, col=1)
fig.update_xaxes(title_text="Time Step", row=1, col=2)
fig.update_yaxes(title_text="Cumulative Return (%)", row=1, col=2)

fig.update_layout(
    title_text=f"Evolution Results - Final Sharpe: {final_metrics[best_final_idx, 5]:.3f}",
    height=500,
    showlegend=True
)

fig.show()

# Summary statistics
print("\n=== Evolution Summary ===")
print(f"Initial Best Fitness: {best_history[0]:.4f}")
print(f"Final Best Fitness: {best_history[-1]:.4f}")
print(f"Improvement: {(best_history[-1] - best_history[0]) / best_history[0] * 100:.1f}%")
print(f"\nFinal Best Individual:")
print(f"  Compound Return: {final_metrics[best_final_idx, 0] * 100:.2f}%")
print(f"  Number of Trades: {int(final_metrics[best_final_idx, 1])}")
print(f"  Win Rate: {final_metrics[best_final_idx, 2] * 100:.1f}%")
print(f"  Sharpe Ratio: {final_metrics[best_final_idx, 5]:.4f}")
print(f"  Calmar Ratio: {final_metrics[best_final_idx, 6]:.4f}")
print(f"  Max Drawdown: {final_metrics[best_final_idx, 4] * 100:.2f}%")


=== Evolution Summary ===
Initial Best Fitness: 0.0000
Final Best Fitness: 8.6749
Improvement: inf%

Final Best Individual:
  Compound Return: 867.49%
  Number of Trades: 33
  Win Rate: 48.5%
  Sharpe Ratio: 103.9267
  Calmar Ratio: 8.6749
  Max Drawdown: 0.00%


/var/folders/82/3qhn3xf53331xztvppn97zc40000gp/T/ipykernel_71021/410196251.py:72: RuntimeWarning:

divide by zero encountered in scalar divide



# Winner's Trading Analysis

In [16]:
# Analyze winner's trades in detail with plotly
best_actions = final_actions[best_final_idx]
returns, n_trades, _ = get_individual_returns_with_costs(best_actions, prices)

# Calculate portfolio value evolution
portfolio_value = np.cumprod(1 + returns) * 100  # Start with $100

# Find trade points
trade_points = np.where(returns != 0)[0]
trade_returns = returns[returns != 0]

# Create detailed visualization with plotly
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=(
        'Price Chart with Trade Signals',
        'Trading Actions Over Time', 
        'Individual Trade Returns',
        'Portfolio Value Evolution'
    ),
    vertical_spacing=0.08,
    row_heights=[0.3, 0.2, 0.2, 0.3]
)

# 1. Price chart with trades
fig.add_trace(
    go.Scatter(x=list(range(len(prices))), y=prices, 
               mode='lines', name='Price',
               line=dict(color='blue', width=1),
               opacity=0.7),
    row=1, col=1
)

# Add buy/sell markers
buy_points = []
sell_points = []
for i in trade_points:
    if best_actions[i] == 2:  # Buy
        buy_points.append(i)
    elif best_actions[i] == 0:  # Sell
        sell_points.append(i)

if buy_points:
    fig.add_trace(
        go.Scatter(x=buy_points, y=[prices[i] for i in buy_points],
                   mode='markers', name='Buy',
                   marker=dict(symbol='triangle-up', size=10, color='green')),
        row=1, col=1
    )

if sell_points:
    fig.add_trace(
        go.Scatter(x=sell_points, y=[prices[i] for i in sell_points],
                   mode='markers', name='Sell',
                   marker=dict(symbol='triangle-down', size=10, color='red')),
        row=1, col=1
    )

# 2. Actions over time with position highlighting
fig.add_trace(
    go.Scatter(x=list(range(len(best_actions))), y=best_actions,
               mode='lines', name='Actions',
               line=dict(shape='hv', width=1)),
    row=2, col=1
)

# Add position periods as background
position = 0
entry_idx = 0
shapes = []
for i in range(len(best_actions)):
    if best_actions[i] == 2 and position == 0:  # Enter long
        position = 1
        entry_idx = i
    elif best_actions[i] == 0 and position == 0:  # Enter short
        position = -1
        entry_idx = i
    elif best_actions[i] == 0 and position == 1:  # Exit long
        shapes.append(dict(
            type="rect", xref=f"x{2}", yref=f"y{2}",
            x0=entry_idx, x1=i, y0=-0.5, y1=2.5,
            fillcolor="green", opacity=0.2, line_width=0
        ))
        position = 0
    elif best_actions[i] == 2 and position == -1:  # Exit short
        shapes.append(dict(
            type="rect", xref=f"x{2}", yref=f"y{2}",
            x0=entry_idx, x1=i, y0=-0.5, y1=2.5,
            fillcolor="red", opacity=0.2, line_width=0
        ))
        position = 0

# 3. Individual trade returns
if len(trade_points) > 0:
    colors = ['green' if r > 0 else 'red' for r in trade_returns]
    fig.add_trace(
        go.Bar(x=trade_points, y=trade_returns * 100,
               marker_color=colors, name='Trade Returns',
               opacity=0.7),
        row=3, col=1
    )

# 4. Portfolio value evolution
fig.add_trace(
    go.Scatter(x=list(range(len(portfolio_value))), y=portfolio_value,
               mode='lines', name='Portfolio Value',
               line=dict(color='green', width=2),
               fill='tozeroy',
               fillcolor='rgba(0,255,0,0.1)'),
    row=4, col=1
)

# Add 100 baseline
fig.add_hline(y=100, line_dash="dash", line_color="black", 
              opacity=0.5, row=4, col=1)

# Update axes
fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="Action", tickvals=[0,1,2], 
                 ticktext=['Sell','Hold','Buy'], row=2, col=1)
fig.update_yaxes(title_text="Return (%)", row=3, col=1)
fig.update_yaxes(title_text="Value ($)", row=4, col=1)
fig.update_xaxes(title_text="Time Step", row=4, col=1)

# Add shapes for position highlighting
fig.update_layout(shapes=shapes)

# Update layout
fig.update_layout(
    title_text=f"Winner's Trading Analysis - Final Value: ${portfolio_value[-1]:.2f}",
    height=1000,
    showlegend=True
)

fig.show()

# Trade statistics
print("=== Winner's Trading Statistics ===")
print(f"Total trades: {n_trades}")
print(f"Average time between trades: {len(best_actions) / n_trades if n_trades > 0 else 0:.1f} steps")

if len(trade_returns) > 0:
    winning_trades = trade_returns[trade_returns > 0]
    losing_trades = trade_returns[trade_returns < 0]
    
    print(f"\nWinning trades: {len(winning_trades)} ({len(winning_trades)/len(trade_returns)*100:.1f}%)")
    if len(winning_trades) > 0:
        print(f"  Average win: {np.mean(winning_trades)*100:.2f}%")
        print(f"  Best win: {np.max(winning_trades)*100:.2f}%")
    
    print(f"\nLosing trades: {len(losing_trades)} ({len(losing_trades)/len(trade_returns)*100:.1f}%)")
    if len(losing_trades) > 0:
        print(f"  Average loss: {np.mean(losing_trades)*100:.2f}%")
        print(f"  Worst loss: {np.min(losing_trades)*100:.2f}%")
    
    print(f"\nProfit factor: {np.sum(winning_trades) / -np.sum(losing_trades) if len(losing_trades) > 0 else np.inf:.2f}")

# Position analysis
long_time = np.sum(best_actions == 2)
short_time = np.sum(best_actions == 0)
flat_time = np.sum(best_actions == 1)

print(f"\nPosition time analysis:")
print(f"  Long: {long_time} steps ({long_time/len(best_actions)*100:.1f}%)")
print(f"  Short: {short_time} steps ({short_time/len(best_actions)*100:.1f}%)")
print(f"  Flat: {flat_time} steps ({flat_time/len(best_actions)*100:.1f}%)")

=== Winner's Trading Statistics ===
Total trades: 33
Average time between trades: 1180.3 steps

Winning trades: 16 (100.0%)
  Average win: 15.25%
  Best win: 16.43%

Losing trades: 0 (0.0%)

Profit factor: inf

Position time analysis:
  Long: 2762 steps (7.1%)
  Short: 25259 steps (64.9%)
  Flat: 10928 steps (28.1%)


In [17]:
prices

array([100.        ,  99.80952071,  99.6235381 , ...,  92.86884801,
        92.87140925,  92.85269645], shape=(38949,))

In [18]:
df_epoch

timestamp,price,message_rate,epoch
i64,f64,f64,i64
3029,100.0,0.016667,0
3510,99.809521,0.033333,0
4616,99.623538,0.05,0
6077,99.43912,0.066667,0
8027,99.270446,0.083333,0
…,…,…,…
29979297,92.88041,0.25,0
29980497,92.864343,0.25,0
29997255,92.868848,0.183333,0


In [19]:
df

timestamp,price,message_rate,epoch
i64,f64,f64,i64
3029,100.0,0.016667,0
3510,99.809521,0.033333,0
4616,99.623538,0.05,0
6077,99.43912,0.066667,0
8027,99.270446,0.083333,0
…,…,…,…
35998080,107.320983,1.883333,1
35998344,107.374834,1.883333,1
35998908,107.406343,1.9,1
